## Modules

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from itertools import product
from pandas import ExcelWriter
from scipy.stats import norm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# `bsoption` module
from bsoption.bsmodel import *

## Single option pricing

### Sample ATM call (with given number of days)

In [2]:
stock1 = 200    # Underlying price
strike1 = 200   # strike price 
numday1 = 45   # number of days to expiry 
sig1 = 50   # IV ( 50% = 0.5)

BStest1 = BSModel(stock1, strike1, numday1, sig1/100)
print(f"Call Price: {BStest1.cprice}")
print(f"Call delta: {BStest1.cdelta}")
print(f"Call vega: {BStest1.vega}")
print(f"Call theta: {BStest1.ctheta}")

Call Price: 13.989829955085767
Call delta: 0.5349745748877144
Call vega: 0.27907868152712395
Call theta: -0.1550437119595133


### Sample OTM put (with expiry date & current date)

In [3]:
stock2 = 315   # Underlying price
strike2 = 300   # strike price 
tradedate2 = datetime(2021, 5, 10)
expiry2 = datetime(2021, 6, 18)
numday2 = (expiry2 - tradedate2).days    # number of days to expiry 
sig2 = 27   # IV 27%


BStest2 = BSModel(stock2, strike2, numday2, sig2/100)
print(f"Put Price: {BStest2.pprice}")
print(f"Put delta: {BStest2.pdelta}")
print(f"Put vega: {BStest2.vega}")
print(f"Put theta: {BStest2.ptheta}")

Put Price: 4.934954358341514
Put delta: -0.27527133493466915
Put vega: 0.34373753899635917
Put theta: -0.11898607119104739


### Payoff diagram: final price only

In [9]:
stock3 = 420  # Underlying price
strike3 = 380   # strike price 
tradedate3 = datetime(2022, 2, 16)
expiry3 = datetime(2022, 3, 30)
numday3 = (expiry3 - tradedate3).days    # number of days to expiry 
sig3 = 24 / 100   # IV 24%

BStest3 = BSModel(stock3, strike3, numday3, sig3/100)
BStest3.getpayoff()

,spot,expC,expP
0,379.5440,0.0000,0.4560
1,379.6352,0.0000,0.3648
2,379.7264,0.0000,0.2736
3,379.8176,0.0000,0.1824
4,379.9088,0.0000,0.0912
5,380.0000,0.0000,0.0000
6,380.0912,0.0912,0.0000
7,380.1824,0.1824,0.0000
8,380.2736,0.2736,0.0000
9,380.3648,0.3648,0.0000


In [8]:
BStest3.getpayoff(preexpiry=True)

NameError: name 'strike' is not defined

## Finding IV by bisection method

### Estimate a single option IV

In [4]:
stock4 = 3500  # Underlying price
strike4 = 4000 # Strike price
tradedate4 = datetime(2022, 4, 4)  # Current trading day
expiry4 = datetime(2022, 4, 29)  # Expiry date
numday4 = (expiry4 - tradedate4).days
opprice4 = 66
optype4 = 'C'

sig4 = getivbisect(stock4, strike4, numday4, opprice4, optype4)
print(f"IV: {sig4}%")

IV: 60.56%


..\bsoption\bsmodel.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  d_1 = (np.log(self.S / self.K) + self.T * (self.rf + 0.5 * (self.sig ** 2))) / (self.sig * (self.T ** 0.5))
..\bsoption\bsmodel.py:31: RuntimeWarning: invalid value encountered in double_scalars
  self.gamma = norm.pdf(self.d_1) / (self.S * self.sig * (self.T ** 0.5))


### Verify IV result by `BSModel()`

In [5]:
BStest4 = BSModel(stock4, strike4, numday4, sig4/100)
print(f"Call Price: {BStest4.cprice}")
print(f"Call delta: {BStest4.cdelta}")
print(f"Call theta: {BStest4.ctheta}")
print(f"Put Price: {BStest4.pprice}")
print(f"Put delta: {BStest4.pdelta}")
print(f"Put theta: {BStest4.ptheta}")
print(f"Option vega: {BStest4.vega}")

Call Price: 65.97451343472005
Call delta: 0.2226537478488897
Call theta: -3.307611598251455
Put Price: 565.9745134347199
Put delta: -0.7773462521511103
Put theta: -3.307611598251455
Option vega: 2.7308550183714124


### Verify Put-Call Parity (PCP)

In [6]:
diffop4 = BStest4.pprice - BStest4.cprice
diffpr4 = strike4 - stock4
print(f'Difference of Put - call: {diffop4}')
print(f'Difference of Strike - Forward: {diffop4}')

Difference of Put - call: 499.9999999999999
Difference of Strike - Forward: 499.9999999999999
